In [1]:
'''
By ryassminh@y
 scraping BCC news articles to build a text-file dataset 
'''
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

In [90]:

url='https://www.bbc.com/sport/football/63809008'
if len(url.split("/"))==6:
    article = requests.get(url)
    soup = BeautifulSoup(article.content, "html.parser")
    body = soup.find("article")
    text=[dd.text.strip() for dd in body.find_all('p')] 
    time=  body.find('time')["datetime"]
    try:
        header=body.find('h1',  attrs={'class':"ssrcss-15xko80-StyledHeading"}).get_text().strip()
    except AttributeError:
        header='None'
    try:
        author=body.find('span',  attrs={'class':"qa-contributor-name"}).get_text().strip('By').strip()
    except AttributeError:
        author='None'
else:
    text=[]
    time=[]
    author=[] 
    header=[]   
print(author)

None


In [123]:
soup.find("article").find('h1').get_text()

"World Cup 2022: Wales' next journey starts with a new identity - Ashley Williams"

In [156]:
soup.find("article").get_text()#.strip('By').strip()

"World Cup 2022: Wales' next journey starts with a new identity - Ashley WilliamsLast updated on 6 hours ago6 hours ago.From the section WalesFifa World CupHost nation: Qatar Dates: 20 November-18 December Coverage: Live on BBC TV, BBC iPlayer, BBC Radio 5 Live, BBC Radio Wales, BBC Radio Cymru, BBC Sounds and the BBC Sport website and app. Day-by-day TV listings - Full coverage detailsWales are going home from this World Cup and it's time to think about what lessons have been learnt for future campaigns, more than what could have been done differently in Qatar.We need to shift our focus to the next major finals - the 2024 European Championship - and decide what direction we will take to get there.That is the target now and I am 100% certain that Rob Page is the man to take us to Germany. He got us to this World Cup, and he knows the squad and what work needs to be done for us to qualify, some of which has been shown in this tournament.Wales need to find an identityThe first thing we n

## Sport news and Future news

In [72]:


# function to extract html document from given url
def getHTMLarticle(url:str) -> list:
    """Parse bbc article and return text in list of strings"""
  
    if len(url.split("/"))==6:
        article = requests.get(url)
        soup = BeautifulSoup(article.content, "html.parser")
        body = soup.find("div",attrs={'class':"qa-story-body"})
        text=[dd.text.strip() for dd in body.find_all('p')] 
        time=  body.find('time')["datetime"]
        try:
            header=body.find('h1',  attrs={'class':"ssrcss-15xko80-StyledHeading"}).get_text().strip()
        except AttributeError:
            header='None'
        try:
            author=body.find('div',  attrs={'class':"ssrcss-68pt20-Text-TextContributorName"}).get_text().strip('By').strip()
        except AttributeError:
            author='None'
    else:
        text=[]
        time=[]
        author=[] 
        header=[]   
    return text,time,author,header

url_input = ['https://www.bbc.com/future',
             'https://www.bbc.com/sport'
             ]

def build_BBC_sport_dataset(seed_urls):
    new_data = []
    for url in seed_urls:
        news_category = url.split('/')[-1]
        data = requests.get(url)
        soup = BeautifulSoup(data.content, 'html.parser')
        #ff=soup.find_all('div',  attrs={'class':"ssrcss-tq7xfh-PromoContent e1f5wbog7"})
        ff=soup.find_all('div',  attrs={'type':"article"})
        for result in ff[:1]:
            article_data={
            #'Header':result.find('h3',{'class':"gs-c-promo-heading__title"}).get_text(strip=True)
            'Header':getHTMLarticle_news(f"https://www.bbc.com{result.find('a').get('href')}")[3]
            ,'Auther':getHTMLarticle_news(f"https://www.bbc.com{result.find('a').get('href')}")[2]
            ,'Article_link':f"https://www.bbc.com{result.find('a').get('href')}"
            ,'Article':getHTMLarticle_news(f"https://www.bbc.com{result.find('a').get('href')}")[0]
            ,'DateTime':getHTMLarticle_news(f"https://www.bbc.com{result.find('a').get('href')}")[1]
            ,'News_category':result.find('a').get('href').split('/')[-3].split('-')[0]
            }
            new_data.extend([article_data]) 
            
    df = pd.DataFrame(new_data)
    #output_dir/outputcsvfile
    df.to_csv('Bcc_sport_articles.csv') 
    return df
   
build_BBC_sport_dataset(url_input)

,Header,Auther,Article_link,Article,DateTime,News_category
0,[],[],https://www.bbc.com/sport/football/63809008,[],[],sport


## Technology Science and Business news

In [66]:
def getHTMLarticle_news(url:str) -> list:
    """Parse bbc article and return text in list of strings"""
  
    if len(url.split("/"))==5:
        article = requests.get(url)
        soup = BeautifulSoup(article.content, "html.parser")
        body = soup.find("article")
        text=[dd.text.strip() for dd in body.find_all('p')]
        time=  body.find('time')["datetime"]
        try:
            header=body.find('h1',  attrs={'class':"ssrcss-15xko80-StyledHeading"}).get_text().strip()
        except AttributeError:
            header='None'
        try:
            author=body.find('div',  attrs={'class':"ssrcss-68pt20-Text-TextContributorName"}).get_text().strip('By').strip()
        except AttributeError:
            author='None'
    else:
        text=[]
        time=[]
        author=[] 
        header=[]   
    return text,time,author,header


url_input = ['https://www.bbc.com/news/technology',
            'https://www.bbc.com/news/science_and_environment',
            'https://www.bbc.com/news/business'
             
             ]

def build_BBC_News_dataset(seed_urls):
    new_data = []
    for url in seed_urls:
        news_category = url.split('/')[-1]
        data = requests.get(url)
        soup = BeautifulSoup(data.content, 'html.parser')
        ff=soup.find_all('div',  attrs={'class':"gs-c-promo-body" })
        for result in ff[:]:
            article_data={
            #'Header':result.find('h3',{'class':"gs-c-promo-heading__title"}).get_text(strip=True)
            'Header':getHTMLarticle_news(f"https://www.bbc.com{result.find('a').get('href')}")[3]
            ,'Auther':getHTMLarticle_news(f"https://www.bbc.com{result.find('a').get('href')}")[2]
            ,'Article_link':f"https://www.bbc.com{result.find('a').get('href')}"
            ,'Article':getHTMLarticle_news(f"https://www.bbc.com{result.find('a').get('href')}")[0]
            ,'DateTime':getHTMLarticle_news(f"https://www.bbc.com{result.find('a').get('href')}")[1]
            ,'News_category':result.find('a').get('href').split('/')[-1].split('-')[0]
            
            }
            new_data.extend([article_data]) 
          
    df = pd.DataFrame(new_data)
    #output_dir/outputcsvfile
    df.to_csv('Bcc_news_articles.csv') 
    return df
   
build_BBC_News_dataset(url_input)  

,Header,Auther,Article_link,Article,DateTime,News_category
0,Plan to make big tech remove harmful content axed,Chris Vallance & Shiona McCallum,https://www.bbc.com/news/technology-63782082,[Controversial measures which would have force...,2022-11-29T01:20:12.000Z,technology
1,Plan to make big tech remove harmful content axed,Chris Vallance & Shiona McCallum,https://www.bbc.com/news/technology-63782082,[Controversial measures which would have force...,2022-11-29T01:20:12.000Z,technology
2,Musk feuds with Apple over Twitter advertising,Ben Derico,https://www.bbc.com/news/technology-63788437,[Elon Musk has said Apple has halted most of i...,2022-11-28T20:41:41.000Z,technology
3,Crypto firm BlockFi files for bankruptcy after...,Natalie Sherman,https://www.bbc.com/news/business-63786201,[The troubled crypto firm BlockFi has filed fo...,2022-11-28T17:20:25.000Z,business
4,Facebook: Meta fined €265m by Irish Data Prote...,None,https://www.bbc.com/news/world-europe-63786893,"[Meta, the company which owns Facebook, Instag...",2022-11-28T17:25:37.000Z,world
...,...,...,...,...,...,...
118,[],[],https://www.bbc.comhttps://www.bbc.com/travel/...,[],[],20221128
119,[],[],https://www.bbc.comhttps://www.bbc.com/culture...,[],[],20221128
120,[],[],https://www.bbc.comhttps://www.bbc.com/future/...,[],[],20221123
121,[],[],https://www.bbc.comhttps://www.bbc.com/worklif...,[],[],20221111
